# Deep Learning for Violence Detection in Surveillance Videos

## Development environment setup

In [ ]:
%%sx
#@title Colab development environment settings
if [ -n "${COLAB_RELEASE_TAG}" ]; then
    git clone -b development https://github.com/MigeoDaSelva/DeepLVioDeSurVideos.git \
    && echo -e "\n\nOkay!\n\n" \
    || echo -e "\n\nFAILED!\n\n";
    python3 -m pip install --upgrade pip;
    python3 -m pip install -r /content/DeepLVioDeSurVideos/requirements.txt;
    kill -9 pid "${PPID}";
fi

In [ ]:
#@title Import libraries
from pprint import pprint
import sys
import os

In [ ]:
#@title System path manager
%reload_ext autoreload
%autoreload 2

project_name = "DeepLVioDeSurVideos"

working_directory = os.popen("echo $PWD").read().rstrip()

project_root_path = f"{working_directory}/{project_name}" \
    if project_name not in working_directory else working_directory

if str(project_root_path) not in sys.path:
    sys.path.insert(0, str(project_root_path))

pprint(f"Environment paths: {sys.path}")

In [ ]:
#@title Import project stuff
from src.application.evaluation_application import EvaluationApplication
from src.application.execution_application import ExecutionApplication
from src.application.startup_application import StartupApplication
from configs import settings

In [ ]:
#@title Colab resource settings
if settings.COLAB_ENV: 
    from google.colab import output
    from google.colab import drive
    output.enable_custom_widget_manager()
    drive.mount("/content/drive", force_remount=True)

In [ ]:
#@title Define simulation scope
simulation_scope_settings = StartupApplication.get_scope_manager()

In [ ]:
#@title Select data settings
data_settings = StartupApplication.get_data_manager()

In [ ]:
#@title Select approach settings
approach_settings = StartupApplication.get_approach_manager()

In [ ]:
#@title Show selected values

print(
    "\nSimulation scope sttings:\n",
    f"Iterations: {simulation_scope_settings.iterations}",
    f"folds: {simulation_scope_settings.folds}",
    sep="\n"
)

print(
    "\nSelected data attributes:\n",
    f"Number of channels: {data_settings.n_channels}",
    f"Number of frames: {data_settings.n_frames}",
    f"Image height: {data_settings.image_height}",
    f"Image width: {data_settings.image_width}",
    f"Batch size: {data_settings.batch_size}",
    sep = "\n"
)

print(
    "\nSelected model attributes:\n",
    f"Callbacks: {[callback.__class__.__name__ for callback in approach_settings.callbacks]}",
    f"Number of output initial filters: {approach_settings.filters}",
    f"Pooling Strides: {approach_settings.pooling_strides}",
    f"Activation function: {approach_settings.activation}",
    f"Padding approach: {approach_settings.padding}",
    f"Kernel size: {approach_settings.kernel_size}",
    f"Number of epochs: {approach_settings.epochs}",
    f"Learning rate: {approach_settings.learning_rate}",
    f"Optimizer: {approach_settings.optimizer}",
    f"Dropout: {approach_settings.dropout}",
    sep = "\n"
)

In [ ]:
#@title build pipeline
pipeline = ExecutionApplication.get_pipeline(
    simulation_scope_settings=simulation_scope_settings,
    data_settings=data_settings,
    approach_settings=approach_settings
)

In [11]:
#@title Run pipeline
pipeline.run()

In [ ]:
#@title Evaluate
evaluation_application = EvaluationApplication(
    approach=pipeline.approach,
    data_settings=data_settings,
    simulation_scope_settings=simulation_scope_settings,
)
evaluation_application.evaluate()

In [ ]:
#@title Calculate and save Precision, Recall and F1-score
evaluation_application.calculate_classification_metrics()

In [ ]:
#@title Calculate AUC
evaluation_application.calculate_auc()

In [ ]:
#@title Build and save ROC Curve
evaluation_application.build_roc_curve()

In [ ]:
#@title Build and save confusion matrix
evaluation_application.build_confusion_matrix()